In [8]:
import json
import os
import sys
from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, callback, dash_table, no_update

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.ingest import aimlabs_task_data_to_dfs, TASKS

In [2]:
fp = Path.cwd().parent / 'data' / 'AimlabTaskData_2023-07-26.04.22.24' / 'taskData.json'

with open(fp, 'r') as f:
    task_data = json.load(f)

data_df, perf_df = aimlabs_task_data_to_dfs(task_data)

In [3]:
for i in data_df.head():
    print(i)
    break

taskId


In [4]:
perf_df.head()
# accuracy = hits/(hits+misses)
# avgDist = ???
# timePerKill
# Irrelevant: (headshots, bodyshots, damageTotal)

,taskId,shotsTotal,hitsTotal,missesTotal,avgDist,damageTotal,timePerKill,killTotal,targetsTotal,accTotal,headshots,bodyshots
0,331,2428,525.0,1903.0,7.899969,0.0,0.0,0,2,21.676300,0,525
1,332,5991,1359.0,4632.0,8.581873,0.0,0.0,0,2,22.684025,0,1359
2,333,5984,1578.0,4406.0,7.816439,0.0,0.0,0,2,26.445452,0,1578
3,334,5998,1984.0,4014.0,11.436398,0.0,0.0,0,39,33.105290,0,1984
4,335,5992,1873.0,4119.0,11.400455,0.0,0.0,0,37,31.258345,0,1873


In [81]:
# draw average / day
# draw smoothed curve
# filters: time/tasknum, score/performance selectors, individual task data?
# select point -> taskID -> performance on radar
#   spider plot scaled to all-time performance?
#   pick non-singleton columns
# deselect points -> show average scalled to all time
app = Dash(__name__)

app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(TASKS, list(TASKS.keys())[0], id='task_name'),
        ], style={'width':'49%', 'display': 'inline-block'}),
        html.Div([
            dcc.RadioItems(['Date', 'Task'], 'Task', id='xaxis_type', inline=True)
        ], style={'width':'49%', 'display': 'inline-block'}),
    ]),

    html.Div([
        # Store range mask
        dcc.Store(id='data-mask'),
        html.Div([
            dcc.Graph(
                id='progression_overview_scatter'
            ),
        ], style={'width':'49%', 'display': 'inline-block'}),
        html.Div([
            dcc.Graph(
                id='task_performance'
            ),
        ], style={'width':'49%', 'float': 'right', 'display': 'inline-block'}),
    ]),
    
    html.Div([
        dash_table.DataTable(id='task_data_table', data=[],
                             page_size=10, fixed_rows={'headers': True},
                             style_table={'overflowY': 'auto'})
    ])
], style={'backgroundColor': 'white'})


@callback(
    # outputs are properties of object with id
    Output('progression_overview_scatter', 'figure'),
    # callbacks are called whenever property changes
    Input('task_name', 'value'),
    Input('xaxis_type', 'value')
)
def plot_scatter(task_name, xaxis_type):
    data_mask = (data_df['taskName'] == TASKS[task_name])
    sub_data = data_df[data_mask]
    if xaxis_type == 'Date':
        fig = px.scatter(sub_data, x='startedAt', y='score') #, custom_data='taskId')
        # can add customdata at the end (must be same size as rest of data)
        fig.update_traces(customdata=data_df[data_mask]['taskId'].to_list())
    elif xaxis_type == 'Task':
        hover_text = [f'{sub_data["taskId"].iloc[i]}: {sub_data["startedAt"].iloc[i]}' for i in range(len(sub_data))]
        fig = go.Figure(
            data=go.Scatter(x=list(range(len(sub_data))),
                            y=sub_data['score'].to_numpy(),
                            mode='markers+lines',
                            hovertext=hover_text,
                            customdata=sub_data['taskId'].to_numpy())
        )
    return fig

@callback(
    # outputs are properties of object with id
    Output('task_performance', 'figure'),
    # callbacks are called whenever property changes
    Input('task_name', 'value'),
    # basic clicks/hover always exist for graphs etc
    Input('progression_overview_scatter', 'clickData')
)
def plot_task_data(task_name, clickData):
    # pass relevant identifier through info via customdata
    try:
        task_id = clickData['points'][0]['customdata']
    except Exception as ex:
        task_id = perf_df['taskId'][0]
    perf_data = get_reduced_task_data(task_id)
    raw_task_data = perf_data[perf_data['taskId'] == task_id]
    raw_task_data = raw_task_data.drop(columns='taskId')
    perf_values = perf_data[perf_data['taskId'] == task_id]
    # normalize value
    perf_values = (perf_values - perf_data.min()) / (perf_data.max() - perf_data.min())
    perf_values = perf_values.drop(columns=['taskId'])
    fig = px.scatter_polar(r=perf_values.values[0], theta=perf_values.columns,
                           hover_data=raw_task_data.values, range_r=[0, 1])
    fig.update_traces(fill='toself')
    fig.update_traces(hovertemplate='Raw Data: %{customdata[0]}')
    # TODo: Scale task data based on time-slice
    # consider selection data, zoom data
    return fig


def get_reduced_task_data(taskId):
    # isolate task type
    task_name = data_df[data_df['taskId']==taskId]['taskName'].iloc[0]
    tasks = data_df[data_df['taskName'] == task_name]
    perfs = perf_df[perf_df['taskId'].isin(tasks['taskId'])]
    # identify invariants
    perfs = perfs.loc[:, (perfs != perfs.iloc[0]).any()]
    # drop component data
    perfs.drop(columns=['shotsTotal', 'hitsTotal', 'missesTotal'], inplace=True)

    return perfs


@callback(
    [Output('task_data_table', 'data'), Output('task_data_table', 'columns')],
    [Input('task_name', 'value')]
)
def display_task_data(task_name):
    sub_data = data_df[data_df['taskName'] == TASKS[task_name]]
    columns = [{"name": i, "id": i} for i in data_df.columns if i != 'performance']
    return sub_data.to_dict('records'), columns




# @callback(
#     Output('task_performance', 'figure'),
#     Input('task_selector', 'task_name')
# )
# def plot_scatter2(task_name):
#     data_mask = (data_df['taskName'] == task_name)
#     fig = px.scatter(data_df[data_mask], x='startedAt', y='score')
#     # print(data_df.head())
#     return fig


In [82]:
app.run(jupyter_mode='inline', debug=True)

In [ ]:
data_df[data_df['taskName'] == list(TASKS.values())[0]].head()

,taskId,taskName,score,mode,performance,startedAt,endedAt
0,331,1_strafe_track,525,42,"{""shotsTotal"":2428,""hitsTotal"":525.0,""missesTo...",2023-07-09 23:37:29,2023-07-09 23:38:55
1,332,1_strafe_track,1359,42,"{""shotsTotal"":5991,""hitsTotal"":1359.0,""missesT...",2023-07-09 23:39:18,2023-07-09 23:40:18
2,333,1_strafe_track,1578,42,"{""shotsTotal"":5984,""hitsTotal"":1578.0,""missesT...",2023-07-09 23:41:49,2023-07-09 23:42:49
24,355,1_strafe_track,1915,42,"{""shotsTotal"":5961,""hitsTotal"":1915.0,""missesT...",2023-07-11 01:41:09,2023-07-11 01:42:09
25,356,1_strafe_track,1743,42,"{""shotsTotal"":5993,""hitsTotal"":1743.0,""missesT...",2023-07-11 01:42:32,2023-07-11 01:43:32


In [58]:
perf_data = get_reduced_task_data(331)
print(perf_data.head())
perf_values = perf_data[perf_data['taskId'] == 331]
print(perf_values)
perf_values = perf_values.drop(columns=['taskId'])
print(perf_values)
# normalize value

    taskId   avgDist   accTotal  bodyshots
0      331  7.899969  21.676300        525
1      332  8.581873  22.684025       1359
2      333  7.816439  26.445452       1578
24     355  8.394752  32.233630       1915
25     356  8.334021  29.156908       1743
   taskId   avgDist  accTotal  bodyshots
0     331  7.899969   21.6763        525
    avgDist  accTotal  bodyshots
0  7.899969   21.6763        525


In [ ]:
TASKS

{'CsLevel.Lowgravity56.VT x WHJ.RWFAP2': '1_strafe_track',
 'CsLevel.Lowgravity56.VT Adjus.ROJF3J': '2_adjust_track',
 'CsLevel.Lowgravity56.VT Berry.RUPVB5': '3_static_entry',
 'CsLevel.Lowgravity56.VT x WHJ.RWIM32': '4_wide_pokeball',
 'CsLevel.Lowgravity56.VT x WHJ.RWFFD8': '5_close_pokeball',
 'CsLevel.VT Empyrean.VT 1w4ts.R1WXON': '6_1w4ts_clusters',
 'CsLevel.Lowgravity56.VT Angle.RVQZXH': '7_dynamic_micro',
 'CsLevel.Lowgravity56.VT x WHJ.RWFGTB': '8_two_pressure'}

None
{'points': [{'curveNumber': 0, 'pointNumber': 13, 'pointIndex': 13, 'x': 13, 'y': 2183, 'hovertext': '429: 2023-07-16 03:51:48', 'bbox': {'x0': 335.49, 'x1': 341.49, 'y0': 180.59, 'y1': 186.59}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 17, 'pointIndex': 17, 'x': 17, 'y': 2033, 'hovertext': '454: 2023-07-20 04:31:55', 'bbox': {'x0': 402.17, 'x1': 408.17, 'y0': 200.23, 'y1': 206.23}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 16, 'pointIndex': 16, 'x': 16, 'y': 2237, 'hovertext': '453: 2023-07-20 04:30:39', 'bbox': {'x0': 385.5, 'x1': 391.5, 'y0': 173.51, 'y1': 179.51}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 11, 'pointIndex': 11, 'x': 11, 'y': 1898, 'hovertext': '406: 2023-07-13 03:17:22', 'bbox': {'x0': 302.15999999999997, 'x1': 308.15999999999997, 'y0': 217.92000000000002, 'y1': 223.92000000000002}}]}
{'points': [{'curveNumber': 0, 'pointNumber': 13, 'pointIndex': 13, 'x': 13, 'y': 2183, 'hovertext': '429: 2023-07-16 03:51:48', 'bbox': {'x0': 335.49, 'x1': 341

In [18]:
task_name = data_df[data_df['taskId']==331]['taskName'][0]
tasks = data_df[data_df['taskName'] == task_name]
# identify invariants
perfs = perf_df[perf_df['taskId'].isin(tasks['taskId'])]

perfs = perfs.loc[:, (perfs != perfs.iloc[0]).any()]

In [19]:
perfs

,taskId,shotsTotal,hitsTotal,missesTotal,avgDist,accTotal,bodyshots
0,331,2428,525.0,1903.0,7.899969,21.676300,525
1,332,5991,1359.0,4632.0,8.581873,22.684025,1359
2,333,5984,1578.0,4406.0,7.816439,26.445452,1578
24,355,5961,1915.0,4046.0,8.394752,32.233630,1915
25,356,5993,1743.0,4250.0,8.334021,29.156908,1743
26,357,5998,1740.0,4258.0,8.360218,29.101858,1740
49,380,5988,1795.0,4193.0,8.534343,30.082119,1795
50,381,5995,1764.0,4231.0,8.366392,29.424520,1764
51,382,5999,1674.0,4325.0,7.989346,27.969923,1674
73,404,5944,1773.0,4171.0,7.828726,29.843462,1773
